## **Problem Statement**:

The goal is to create an encoder-decoder model with attention mechanism for **sentiment analysis**. The model should be able to understand the meaning and emotions of a given input sentence and output a ***sentiment score between -1 (negative) and 1 (positive), with 0 being neutral. This can be used in call centers to analyze the evolution of a client's emotions and their reactions to certain keywords or company discounts.

Solution:
We'll be using the TensorFlow library to create our sentiment analysis model. In this example, we will be using the IMDB movie review dataset to train and test our model.


### **Steps Involved**

> **Import necessary libraries and modules**:
Import NumPy, TensorFlow, and other necessary modules for creating and training the model.

> **Load and preprocess the dataset**:
Load the IMDB dataset and preprocess it by padding the sequences and scaling the target values between -1 and 1.

> **Define the model parameters**:
Set the vocabulary size, embedding size, and LSTM units for the model.

> **Build the encoder**:
Create an input layer, an embedding layer, and a bidirectional LSTM layer for the encoder.

> **Build the decoder**:
Create an LSTM layer for the decoder, using the states from the encoder as initial states.

>**Implement the attention mechanism**:
Add an additive attention layer and a global average pooling layer to combine the outputs from the encoder and decoder.

> **Create the output layer**:
Add a dense layer with a 'tanh' activation function to output the sentiment score.

> **Compile the model**:
Compile the model using the 'adam' optimizer, 'mean_squared_error' loss function, and 'accuracy' as a metric.

> **Train the model**:
Train the model on the IMDB dataset for 5 epochs with a batch size of 32 and using validation data.

> **Create the sentiment_analysis function**:
Define a function that takes an input text, tokenizes it, pads it, and feeds it to the trained model to get the sentiment score.

> **Test the model**:
Test the sentiment_analysis function with an example text and print the sentiment score.

These steps guide you through the process of building, training, and testing an encoder-decoder model with an attention mechanism for sentiment analysis.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Bidirectional, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load and preprocess the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=5000)
max_len = 200
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)
y_train = y_train * 2 - 1
y_test = y_test * 2 - 1

# Model parameters
vocab_size = 5000
embedding_size = 128
lstm_units = 64

# Encoder
encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(vocab_size, embedding_size, mask_zero=True)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])

# Decoder
decoder_lstm = LSTM(lstm_units * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(encoder_outputs, initial_state=[state_h, state_c])

# Attention mechanism
attention = tf.keras.layers.AdditiveAttention()([decoder_outputs, encoder_outputs])
context_vector = tf.keras.layers.GlobalAveragePooling1D()(attention)

# Output layer
output = Dense(1, activation='tanh')(context_vector)

# Compile the model
model = Model(encoder_inputs, output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test))

# Sentiment analysis function
def sentiment_analysis(text):
    tokenizer = Tokenizer(num_words=vocab_size)
    input_seq = tokenizer.texts_to_sequences([text])
    input_seq = pad_sequences(input_seq, maxlen=max_len)
    sentiment_score = model.predict(input_seq)
    return sentiment_score

# Test the model
text = "The customer service was excellent, and I loved the discount!"
sentiment_score = sentiment_analysis(text)
print("Sentiment Score:", sentiment_score)


17464789/17464789 [==============================] - 0s 0us/step
Epoch 1/5
  5/782 [..............................] - ETA: 1:00:39 - loss: 1.0035 - accuracy: 0.0000e+00